<a href="https://colab.research.google.com/github/amalolan/COVID19_Tweets_Dataset/blob/master/Automatically_Hydrate_TweetsIDs_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




<center><b>© 2020. Content is made available under the CC-BY-NC-ND 4.0 license. Christian Lopez, lopezbec@lafayette.edu/  Malolan Vasu, vasum@lafayette.edu <b><center>

**UPDATED ON 4/17/2020**


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lopezbec/COVID19_Tweets_Dataset/blob/master/Automatically_Hydrate_TweetsIDs_COVID19.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# **Notebook to automatically "hydrate" tweets-ID**

This notebook will allow you to automatically hydrate the tweets-ID from our [COVID19_Tweets_dataset GitHub repository](https://github.com/lopezbec/COVID19_Tweets_Dataset).


You can run this notebook directly on the cloud using Google Colab [(see how to tutorials)]( https://colab.research.google.com/notebooks/welcome.ipynb#scrollTo=xitplqMNk_Hc) and Google Drive.

In order to hydrate the tweet-IDs using [TWARC](https://github.com/DocNow/twarc) you need to create a [Twitter Developer Account]( https://developer.twitter.com/en/apply-for-access).

First run the Initialization block to set up the required libraries. Then, run the Configuration block and choose your required configuration for tweet IDs to hydrate (dates needed, keywords needed). Finally, run the Hydrate block.  In the case that the code unexpectedly stops in the hydrate phase, you only need to run Initialization and Hydration. You aren't required to run configuration.


# Initialization

### Mount Drive

The code will clone the repository and place it in your Google drive. Here you need to type where in your Google Drive you would like the information stored

In [0]:
#@title Set up Directory { run: "auto"}
import os
from IPython.display import clear_output
from google.colab import drive 
from IPython.display import clear_output
drive.mount('/content/gdrive')
working_directory = 'My Drive/Research/form' #@param {type:"string"}
wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
current directory is : /content/gdrive/My Drive/Research/form


### Install twarc

In [0]:
%pip install twarc
%pip install jsonlines
clear_output()

In [0]:
#Check if TWARC was installed correctly on the Virtual Machine
%pip show twarc
%pip show jsonlines

### Twitter API Keys

In [0]:
#@title Insert API Keys here { run : "auto"}
from twarc import Twarc

# These keys are received after applying for a twitter developer account

consumer_key = "" #@param {type:"string"}
consumer_secret = "" #@param {type:"string"}
access_token = "" #@param {type:"string"}
access_token_secret = "" #@param {type:"string"}

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

### Clone Github Repository onto Drive

This will clone the repository and place it in your Google drive. If the repository has already been cloned, it will update the repo and pull changes.

In [0]:
!if cd COVID19_Tweets_Dataset; then git pull; else git clone https://github.com/lopezbec/COVID19_Tweets_Dataset.git COVID19_Tweets_Dataset; fi

# Configuration: Choose Settings

### Keywords

We recommend you run the code for a few keywords and create many output.csv files. Then, you can copy them to a different directory, and merge them when  required. Selecting too many keywords and dates might result in data too large to be stored in memory. 

In [0]:
#@title Check Keywords to Hydrate { run: "auto" }
coronavirus = True #@param {type:"boolean"}
virus = False #@param {type:"boolean"}
covid = False #@param {type:"boolean"}
ncov19 = False #@param {type:"boolean"}
ncov2019 = False #@param {type:"boolean"}
keyword_dict = {"coronavirus": coronavirus, "virus": virus, "covid": covid, "ncov19": ncov19, "ncov2019": ncov2019}

### Get Number of Tweets by Dates

If you are running this in Google Colab we recommend starting with a range of just 1 day, do to the Google Colab 12hr limit and Twitter API limit. 

In [0]:
#@title Enter range of dates to Hydrate { run: "auto" }
start_date = '2020-01-22' #@param {type:"date"}
end_date = '2020-01-23' #@param {type:"date"}


import datetime as dt
files = []
covid_loc = "COVID19_Tweets_Dataset"
# Looks at each volder
for folder in os.listdir(covid_loc):
    foldername = os.fsdecode(folder)
    # The folder name is a keyword. We continue for keywords selected above
    if keyword_dict.get(foldername.split()[0].lower()) == True:
        folderpath = os.path.join(covid_loc, foldername)
        # Each file is of the format [keyword]_yyyy_mm_dd.txt
        for file in os.listdir(folderpath):
            filename = os.fsdecode(file)
            date = filename[filename.index("_")+1:filename.index(".")]

            # If the date is within the required range, it is added to the
            # list of files to read.
            if (dt.datetime.strptime(start_date, "%Y-%m-%d").date() 
            <= dt.datetime.strptime(date, '%Y_%m_%d').date()
             <= dt.datetime.strptime(end_date, "%Y-%m-%d").date()):
                files.append(os.path.join(folderpath, filename))
# The final list is read, and each of the individual IDs is stored in a collective
# set of IDs. Duplicates are removed.
ids = set()
for filename in files:
    with open(filename) as f:
        # The files are of the format: [id1,id2,id3,...,idn]
        # Remove the brackets and split on commas
        for i in f.readline().strip('][').replace(" ", "").split(","):
            ids.add(i) 
# Number of tweets read.
print(round((len(ids)/1000000), 3), "million unique tweets.")

### Save configuration into a file
All the IDs are read into a single set in the previous code block using the specified configuration. The ID Output file stores all the IDs in a single file so that the configuration blocks don't have to be run again. In case the program unexpectedly stops, you can just run the code for Initialization and then the code for Hydration.

In [0]:
#@title Enter ID output file {run: "auto"}
final_tweet_ids_filename = "final_ids.txt" #@param {type: "string"}
# The set of IDs is stored in this file.
with open(final_tweet_ids_filename, "w+") as f:
    for id in ids:
        f.write('%s\n' % id)

# Hydrate

### Set up output file

The final_tweet_ids_filename should be exactly the same as the ID output file from the Configuration block. If this file does not exist in the working directory, you have to re-run the Configuration block.

Please also keep the output_filename the same in case the code is halted. That way, tweets already hydrated aren't re-hydrated for no reason. 

Also, please do not remove the .txt file created after running the Hydrate block until all the data is converted to a CSV file.

In [0]:
#@title Set up Directory { run: "auto"}
final_tweet_ids_filename = "final_ids.txt" #@param {type: "string"}
output_filename = "output.csv" #@param {type: "string"}

The time for this code will depend on how many tweets you want to “hydrate”. Also, be advise of the Tweet API limit, the code will “go to sleep” once the limit is reach and automatically continue. 
You can leave this code running in Google Colab for a max of 12hrs. 


In [1]:
import jsonlines, json
# Stores hydrated tweets here as jsonl objects
# Contains one json object per line
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
ids = []
with open(final_tweet_ids_filename, "r") as ids_file:
    ids = ids_file.read().split()
hydrated_tweets = []
ids_to_hydrate = set(ids)

# Looks at the output file for already hydrated tweets
if os.path.isfile(output_json_filename):
    with jsonlines.open(output_json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            # These tweets have already been hydrated. So remove them from ids_to_hydrate
            hydrated_tweets.append(i)
            ids_to_hydrate.remove(i["id_str"])
print("Total IDs: " + str(len(ids)) + ", IDs to hydrate: " + str(len(ids_to_hydrate)))
print("Hydrated: " + str(len(hydrated_tweets)))

count = len(hydrated_tweets)
start_index = count # The index from where tweets haven't been saved to the output_json_file
# Stores hydrated tweets to output_json_file every num_save iterations.
num_save  = 1000

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate):
    hydrated_tweets.append(tweet)
    count += 1
    # If num_save iterations have passed,
    if (count % num_save) == 0:
        # Open the output file
        # NOTE: Even if the code stops during IO, only tweets from the current iteration are lost.
        # Older tweets are preserved as the file is written in append mode.
        with jsonlines.open(output_json_filename, "a") as writer:
            print("Started IO")
            # Now write the tweets from start_index. The other tweets don't have to be written
            # as they were already written in a previous iteration or run.
            for hydrated_tweet in hydrated_tweets[start_index:]:
                writer.write(hydrated_tweet)
            print("Finished IO")
        print("Saved " + str(count) + " hydrated tweets.")
        # Now, since everything has been written. Reset start_index
        start_index = count
# There might be tweets unwritten in the last iteration if the count is not a multiple of num_tweets.
# In that case, just write out the remainder of tweets.
if count != start_index:
    print("Here with start_index", start_index)
    with jsonlines.open(output_json_filename, "a") as writer:
        for hydrated_tweet in hydrated_tweets[start_index:]:
           writer.write(hydrated_tweet)   

ModuleNotFoundError: ignored

## Convert JSONL to CSV
Data is stored in  output_json_file from the previous code block. This now converts the jsonl .txt file into a csv file. Note that the column names required is stored as a list in the code.

Note that a few of the columns are actually json objects (for example, user or entities). You will have to clean these objects into 1D data.

In [0]:
# Convert jsonl to csv
import csv, jsonlines
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
# These are the column name that are selected to be stored in the csv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]
hydrated_tweets = []
# Reads the current tweets
with jsonlines.open(output_json_filename, "r") as reader:
    for i in reader.iter(type=dict, skip_invalid=True):
        hydrated_tweets.append(i)
# Writes them out
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)

Your data is now stored in output_filename. If you want to re-run  this notebook, please copy this file to a different directory.